# <u> Reset Test Environment </u>

This notebook will use C#, SQL and PowerShell


Use this notebook to reset my test environment for the "Practical Starter Guide to SQL Notebooks" session.

<b>NOTE:</b> This works best if no tools are open and this tool has everything closed as well.

Pull down the .NET interactive packages for connecting to SQL Server before starting:

to pull down the latest, use #r "nuget:Microsoft.DotNet.Interactive.SqlServer,*-*"

In [1]:
#r "nuget: Microsoft.DotNet.Interactive.SqlServer,*-*"

Installed Packages Microsoft.DotNet.Interactive.SqlServer, 1.0.0-beta.23152.2

Loading extensions from `C:\Users\Deborah\.nuget\packages\microsoft.dotnet.interactive.sqlserver\1.0.0-beta.23152.2\interactive-extensions\dotnet\Microsoft.DotNet.Interactive.SqlServer.dll`

Query Microsoft SQL Server databases. 
 This extension adds support for connecting to Microsoft SQL Server databases using the #!connect mssql magic command. For more information, run a cell using the #!sql magic command.

## <u> SQL Server clean up:</u>
* Drop the SuperheroesPublish database if it exists
* Restore the Superheroes back to the original
* Drop the extended events session if it exists

NOTE: If you're curious as to why I'm turning off encryption - https://github.com/dotnet/SqlClient/issues/1836. 

<i>Remember this is a test and only a test.</i> 
For production - make sure you have encryption for connection strings set up properly.

In [2]:
#!connect mssql --kernel-name mydatabase "Persist Security Info=False; Integrated Security=true; Encrypt=False; Initial Catalog=master; Server=debthedba\sql2019"

Kernel added: #!sql-mydatabase

In [3]:
#!sql-mydatabase
IF EXISTS (SELECT * FROM sys.databases WHERE Name = 'SuperheroesPublish')
DROP DATABASE SuperheroesPublish

Commands completed successfully.

In [5]:
#!sql-mydatabase

DECLARE @session varchar(30), @backupfile nvarchar(300), @today DATE = getdate()

SELECT @Session = 'PracticalStarterGuide'
SELECT @backupfile = 'C:\GitRepo\SQL_Presentations\Superhero Demo DB Setup\Superheroes_SQL2019_' + @Session + '_Initial.BAK'
SELECT @backupfile

DECLARE @backuplogname nvarchar(50) = 'Superheroes_LogBackup' + convert(varchar(10), @today) + '.BAK'

RESTORE DATABASE [Superheroes] 
FROM  DISK = @backupfile WITH  FILE = 1,  NOUNLOAD,  REPLACE,  STATS = 5


(1 row affected)

5 percent processed.

10 percent processed.

15 percent processed.

20 percent processed.

...

100 percent processed.

Processed 640 pages for database 'Superheroes', file 'Superheroes' on file 1.

Processed 1 pages for database 'Superheroes', file 'Superheroes_log' on file 1.

RESTORE DATABASE successfully processed 641 pages in 0.088 seconds (56.834 MB/sec).

(No column name)
C:\GitRepo\SQL_Presentations\Superhero Demo DB Setup\Superheroes_SQL2019_PracticalStarterGuide_Initial.BAK


In [6]:
#!sql-mydatabase

IF EXISTS (SELECT * FROM sys.server_event_sessions WHERE name = 'Summit2021')
DROP EVENT SESSION [Summit2021] ON SERVER 
GO

Commands completed successfully.

## <u> Directory Clean Up: </u>

Clean up directories
* Delete the database project directory 
* Remove the docker container


In [7]:
$dbprojlocation = "C:\GitRepo\SQL_Presentations\A Practical Starter Guide to SQL Notebooks"

Get-ChildItem -Path $dbprojlocation | WHERE {$_.Name -like 'Superheroes*'} | ForEach-Object {
    $name = $_.FullName

    write-host "Removing item $name ..."
    Remove-Item $name -Recurse -Force
}

Removing item C:\GitRepo\SQL_Presentations\A Practical Starter Guide to SQL Notebooks\SuperheroesDacPac ...


In [8]:
docker container stop PracticalStarterGuide
docker rm PracticalStarterGuide

PracticalStarterGuide
PracticalStarterGuide
